In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
##########################
save_history = False # if true, updates portfolio and platform history files.
##########################
today = pd.to_datetime('today').normalize()
loans_df = pd.read_csv('data/concatenated.csv')
cash_df = pd.read_csv('data/cash.csv', index_col='platform_account_name')
transactions_df = pd.read_csv('data/transactions.csv')
#platform_dict = { 'loanpad':'Loanpad 60d Access','proplend':'Proplend' }
platform_dict = { 'loanpad':'Loanpad 60d Access','proplend':'Proplend','crowdproperty':'CrowdProperty AutoInvest','invest_fund':'Invest & Fund','assetz_capital':'Assetz Capital 90d Access','kuflink':'Kuflink 24m Term 28133' }
col_list = ['Paid in capital','Distr capital','Int bearing','Non bearing','Accr int','Residual value','DPI','RVPI','TVPI','Inv mnths','TVPI CAGR','Net AER','Blended LTV','No loans','Avg loan amt','Avg term','Max term','Avg ttm','Max ttm','Avg ext','Ext 90d','Non-perform','Long default','Short default','Int default']
format_dict = { 'Paid in capital': '£{0:,.2f}','Distr capital': '£{0:,.2f}','Int bearing': '£{0:,.2f}','Non bearing': '£{0:,.2f}','Accr int': '£{0:,.2f}','Residual value': '£{0:,.2f}','DPI': '{0:,.2f}', 'RVPI': '{0:,.2f}', 'TVPI' : '{0:,.2f}','Inv mnths': '{:,.0f}','TVPI CAGR': '{:,.1%}','Net AER': '{:,.1%}','No loans': '{:,.0f}','Avg loan amt': '£{0:,.2f}','Blended LTV': '{:,.0%}','Avg term': '{:,.0f}','Max term': '{:,.0f}','Avg ttm': '{:,.0f}','Max ttm': '{:,.0f}','Avg ext': '{:,.0f}','Ext 90d': '{:,.0%}','Non-perform': '{:,.0%}','Long default': '£{0:,.0f}','Short default': '£{0:,.0f}','Int default': '£{0:,.0f}','Dev sector': '{:.0%}', 'Comm sector': '{:.0%}','No a/c': '{:,.0f}','Avg a/c amt': '£{0:,.0f}'}
colours_list = ['green','blue','red','orange','purple','pink']
def save_platform_csv(platform): # function to update platform_history.csv for all platform accounts
    exec(platform + "_df = pd.read_csv('data/" + platform + "_history.csv', index_col=0, parse_dates=[0])") # read platform _history.csv
    exec(platform + "_df.loc[today] = platform_row") # append platform row
    exec(platform + "_df.to_csv('data/" + platform + "_history.csv', index=True)") # save platform _history.csv

### Platforms

In [3]:
platform_df = pd.DataFrame(columns=col_list)
for platform in platform_dict:
    platform_account_name = platform_dict[platform]
    platform_loans = loans_df.loc[(loans_df['platform_account_name'] == platform_account_name) & (loans_df['loan_part_status'] != 'Repaid')]
    platform_int_bearing = platform_loans['loan_part_amt'].sum() + cash_df.loc[platform_account_name]['ib_cash'].sum()
    platform_non_bearing = cash_df.loc[platform_account_name]['nonib_cash']
    platform_accrued_int = cash_df.loc[platform_account_name]['accrued_int']
    platform_residual_value = platform_int_bearing + platform_non_bearing + platform_accrued_int
    platform_extended_pct = platform_loans.loc[platform_loans['loan_part_maturity_extended_days'] > 90, 'loan_part_amt'].sum() / platform_loans['loan_part_amt'].sum()
    platform_nonperform_pct = platform_loans.loc[platform_loans['loan_part_status'].isin(['Short Default', 'Long Default', 'Interest Default']), 'loan_part_amt'].sum() / platform_loans['loan_part_amt'].sum()
    platform_avg_loan_amt = platform_loans['loan_part_amt'].sum() / platform_loans['loan_portion'].sum()
    platform_transactions = transactions_df.loc[transactions_df['platform_account_name'] == platform_account_name]
    platform_paid_in = platform_transactions.loc[platform_transactions['trx_type'] == 'Paid in', 'trx_amt'].abs().sum()
    platform_distributions = platform_transactions.loc[platform_transactions['trx_type'] == 'Distribution', 'trx_amt'].abs().sum()
    platform_dpi = np.where(platform_distributions != 0, platform_distributions/platform_paid_in, 0)
    platform_rvpi = platform_residual_value / platform_paid_in
    platform_invested_months = (pd.to_datetime('today').date() - pd.to_datetime(platform_transactions['trx_date'].min()).date()).days / 30.437
    platform_cagr = (platform_dpi + platform_rvpi)**(1/(platform_invested_months/12))-1
    platform_row = [platform_paid_in, platform_distributions, platform_int_bearing, platform_non_bearing, platform_accrued_int, platform_residual_value, platform_dpi, platform_rvpi, platform_dpi + platform_rvpi, platform_invested_months, platform_cagr, platform_loans['platform_weighted_net_aer'].sum(), platform_loans['platform_weighted_blended_ltv'].sum(), platform_loans['loan_portion'].sum(), platform_avg_loan_amt, platform_loans['platform_weighted_term_months'].sum(), platform_loans['loan_part_term_months'].max(), platform_loans['platform_weighted_ttm_months'].sum(), platform_loans['loan_part_ttm_days'].max()/33.437, platform_loans['platform_weighted_maturity_extended_days'].sum(), platform_extended_pct, platform_nonperform_pct, platform_loans.loc[platform_loans['loan_part_status'] == 'Long Default', 'loan_part_amt'].sum(),platform_loans.loc[platform_loans['loan_part_status'] == 'Short Default', 'loan_part_amt'].sum(),platform_loans.loc[platform_loans['loan_part_status'] == 'Interest Default', 'loan_part_amt'].sum()]
    if save_history == True:
        save_platform_csv(platform) # function to update platform_history.csv for all platform accounts
    platform_df.loc[platform_account_name] = platform_row
platform_df.sort_values(by='Residual value', ascending=False).style.format(format_dict)

ValueError: Can only compare identically-labeled Series objects

In [ ]:
portfolio_df = pd.read_csv('data/portfolio_history.csv', index_col=0, parse_dates=[0])
portfolio_paid_in = transactions_df.loc[transactions_df['trx_type'] == 'Paid in', 'trx_amt'].abs().sum()
portfolio_distributions = transactions_df.loc[transactions_df['trx_type'] == 'Distribution', 'trx_amt'].abs().sum()
portfolio_accrued_int = cash_df['accrued_int'].sum()
portfolio_int_bearing = loans_df['loan_part_amt'].sum() + cash_df['ib_cash'].sum()
portfolio_non_bearing = cash_df['nonib_cash'].sum()
portfolio_accrued_int = cash_df['accrued_int'].sum()
portfolio_residual_value = portfolio_int_bearing + portfolio_non_bearing + portfolio_accrued_int
portfolio_dpi = portfolio_distributions / portfolio_paid_in
portfolio_rvpi = portfolio_residual_value / portfolio_paid_in
portfolio_invested_months = (pd.to_datetime('today').date() - pd.to_datetime(transactions_df['trx_date'].min()).date()).days / 30.437
portfolio_cagr = (portfolio_dpi + portfolio_rvpi)**(1/(portfolio_invested_months/12))-1
portfolio_row = [portfolio_paid_in, portfolio_distributions, portfolio_int_bearing, portfolio_non_bearing, portfolio_accrued_int, portfolio_residual_value, portfolio_dpi, portfolio_rvpi, portfolio_dpi + portfolio_rvpi, portfolio_invested_months, portfolio_cagr, loans_df['portfolio_weighted_net_aer'].sum(), loans_df['portfolio_weighted_blended_ltv'].sum(), loans_df['loan_portion'].sum(), loans_df['loan_part_amt'].sum() / loans_df['loan_portion'].sum(), loans_df['portfolio_weighted_term_months'].sum(),loans_df['loan_part_term_months'].max(),loans_df['portfolio_weighted_ttm_months'].sum(),loans_df['loan_part_ttm_days'].max()/33.437,loans_df['portfolio_weighted_maturity_extended_days'].sum(),loans_df.loc[loans_df['loan_part_maturity_extended_days'] > 90, 'loan_part_amt'].sum() / loans_df['loan_part_amt'].sum(),loans_df.loc[loans_df['loan_part_status'].isin(['Short Default', 'Long Default', 'Interest Default']), 'loan_part_amt'].sum() / loans_df['loan_part_amt'].sum(),loans_df.loc[loans_df['loan_part_status'] == 'Long Default', 'loan_part_amt'].sum(),loans_df.loc[loans_df['loan_part_status'] == 'Short Default', 'loan_part_amt'].sum(),loans_df.loc[loans_df['loan_part_status'] == 'Interest Default', 'loan_part_amt'].sum(), loans_df.loc[loans_df['platform_account_sector'] == 'Development']['loan_part_amt'].sum()/loans_df['loan_part_amt'].sum(),loans_df.loc[loans_df['platform_account_sector'] == 'Commercial']['loan_part_amt'].sum()/loans_df['loan_part_amt'].sum(),len(platform_dict),loans_df['loan_part_amt'].sum() / len(platform_dict),]
portfolio_df.loc[today] = portfolio_row
if save_history == True:
    portfolio_df.to_csv('data/portfolio_history.csv', index=True)
portfolio_df.sort_index(ascending=False).style.format_index(lambda s: s.strftime('%Y-%m')).format(format_dict)

In [ ]:
ax = platform_df.plot.barh(y='Avg loan amt', legend=False)
ax.bar_label(ax.containers[0], fmt='£{0:,.2f}', label_type='edge', padding=5)

In [ ]:
ax = platform_df['Residual value'].plot(kind='pie', colors=colours_list, autopct='%1.0f%%', legend=False)

In [ ]:
ax = platform_df.plot(kind='scatter', x='Blended LTV', y='Net AER', s=150, c=colours_list)